In [ ]:
# Importing important libraries 
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from gensim.summarization import keywords
from gensim.parsing.preprocessing import remove_stopwords
import re

In [177]:
news_count_threshold = 900
sub_cat_dict = {"india-news": ["politics", "law-and-order", "economy"], 
                "business-news": ["international-business", "india-business"], 
                "technology-news": ["gadgets", "mobile", "other-tech-news", "gaming"], 
                "sports-news": ["ipl-2022", "football-news", "cricket-news"], 
                "entertainment-news": ["bollywood-news", "hollywood-news", "television-news", "regional-indian-cinema"]}

In [179]:
total_cat_count = len(sub_cat_dict)
total_sub_cat_count = len([i for v in sub_cat_dict.values() for i in v])
news_per_page_count = 12
news_per_category = int(news_count_threshold/total_sub_cat_count)
pages_per_sub_cat = 1+ int(news_per_category/news_per_page_count)
news_headlines = list()
news_links = list()
news_authors = list()
news_dates = list()
news_times = list()
news_entire = list()
news_cats = list()
news_subcats = list()

In [182]:
news_per_category*16

896

In [14]:
cat = 'india-news'
sub_cat= "politics"
page_no = 5
url = "https://www.republicworld.com/{}/{}/{}".format(cat,sub_cat, page_no)
                    # Using BeautifulSoup for web crawling
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")


In [224]:
total_cat_count = len(sub_cat_dict)
total_sub_cat_count = len([i for v in sub_cat_dict.values() for i in v])
news_per_page_count = 12
news_per_category = int(news_count_threshold/total_sub_cat_count)
pages_per_sub_cat = 2 + int(news_per_category/news_per_page_count)
news_headlines = list()
news_links = list()
news_authors = list()
news_dates = list()
news_times = list()
news_entire = list()
news_cats = list()
news_subcats = list()


for cat in sub_cat_dict:
    for sub_cat in sub_cat_dict[cat]:
        print(str(cat)+':'+str(sub_cat))
        for page_no in range(1,pages_per_sub_cat):
          url = "https://www.republicworld.com/{}/{}/{}".format(cat,sub_cat, page_no)
          page = requests.get(url)
          soup = BeautifulSoup(page.content, "html.parser")
          for i in soup.find_all('article',class_="channel-card hover-effect"):
            news_links.append(i.a.get('href'))
            p = requests.get(news_links[-1],allow_redirects=False)
            s = BeautifulSoup(p.content, "html.parser")
            try:
              text =''
              para = s.find("div",class_='width100 storytext').findAll('p')
              for element in para:
                text += ''.join(element.findAll(text = True))
              headline = s.find('h1',class_='story-title').get_text().strip()
              author = s.find('div',class_='author-title').span.get_text().strip()
              date = s.time.attrs['datetime'].split('T')[0]
              time = s.time.attrs['datetime'].split('T')[1].split('+')[0]
            except:
              text = "text not available"
              headline = "headline not available"
              author = 'author not available'
              date = 'date not available'
              time = 'time not available'




            news_authors.append(author)
            news_headlines.append(headline)
            news_dates.append(date)
            news_times.append(time)
            news_entire.append(text.strip())
            news_cats.append(cat)
            news_subcats.append(sub_cat)

india-news:politics
india-news:law-and-order
india-news:economy
business-news:international-business
business-news:india-business
technology-news:gadgets
technology-news:mobile
technology-news:other-tech-news
technology-news:gaming
sports-news:ipl-2022
sports-news:football-news
sports-news:cricket-news
entertainment-news:bollywood-news
entertainment-news:hollywood-news
entertainment-news:television-news
entertainment-news:regional-indian-cinema


In [154]:
i.a.get('href')

'https://www.republicworld.com/sports-news/cricket-news/icc-womens-odi-rankings-mithali-raj-and-jhulan-goswami-rise-in-latest-rankings-articleshow.html'

In [225]:
len(news_dates),len(news_times),len(news_cats),len(news_subcats),len(news_headlines),len(news_entire),len(news_authors),len(news_links)

(960, 960, 960, 960, 960, 960, 960, 960)

In [231]:
News_Corpus = pd.DataFrame({"Date": news_dates, "Time": news_times, "Category": news_cats,
                            "Sub-Category": news_subcats, "Headline": news_headlines,
                           "Entire_News": news_entire, "Author": news_authors, "News_Link": news_links})

News_Corpus = News_Corpus[News_Corpus['Entire_News'].str.contains('Coin Master')==False]
News_Corpus = News_Corpus[News_Corpus['Entire_News']!='Get the latest entertainment news from India & around the world. Now follow your favourite television celebs and telly updates. Republic World is your one-stop destination for trending Bollywood news. Tune in today to stay updated with all the latest news and headlines from the world of entertainment.']

In [232]:
News_Corpus

,Date,Time,Category,Sub-Category,Headline,Entire_News,Author,News_Link
0,2022-04-05,16:58:33,india-news,politics,Bill To Ban Financing WMDs Introduced In LS,"New Delhi, Apr 5 (PTI) The government on Tuesd...",Press Trust Of India,https://www.republicworld.com/india-news/polit...
1,2022-04-05,16:51:03,india-news,politics,No One Can Erase My Name From Nagpur-Mumbai Ex...,"Nagpur, Apr 5 (PTI) Maharashtra BJP leader De...",Press Trust Of India,https://www.republicworld.com/india-news/polit...
2,2022-04-05,16:47:40,india-news,politics,Congress Faces Big Blow As Ahmed Patel's Son F...,"In a massive development, Republic has now lea...",Vishnu V V,https://www.republicworld.com/india-news/polit...
3,2022-04-05,16:32:39,india-news,politics,Congress To Fight On Its Own In Telangana; Rah...,Congress will not ally with the Telangana Rash...,Abhishek Raval,https://www.republicworld.com/india-news/polit...
4,2022-04-05,16:29:34,india-news,politics,Parliament Approves Accountancy Bill,"New Delhi, Apr 5 (PTI) Parliament on Tuesday a...",Press Trust Of India,https://www.republicworld.com/india-news/polit...
...,...,...,...,...,...,...,...,...
955,2022-03-27,18:53:32,entertainment-news,regional-indian-cinema,'Vaashi': Tovino Thomas & Keerthy Suresh Tease...,After piquing fans' excitement with their firs...,Kriti Nayyar,https://www.republicworld.com/entertainment-ne...
956,2022-03-27,17:10:37,entertainment-news,regional-indian-cinema,Ram Charan's Wife Upasana Throws Confetti At S...,Celebrated filmmaker SS Rajamouli's critically...,Swati Singh,https://www.republicworld.com/entertainment-ne...
957,2022-03-27,16:56:03,entertainment-news,regional-indian-cinema,Chiranjeevi Posts Sweet Childhood Pic Of Ram C...,"Ram Charan, who is currently basking in the su...",Adelle Fernandes,https://www.republicworld.com/entertainment-ne...
958,2022-03-27,16:07:51,entertainment-news,regional-indian-cinema,'KGF: Chapter 2' Trailer: Here's How Fans Are ...,"After a wait of three years, fans are all set ...",Aditi Rathi,https://www.republicworld.com/entertainment-ne...


In [233]:
len(News_Corpus)

941

In [239]:
News_Corpus.to_csv('Republic.csv')

In [238]:
News_Corpus[News_Corpus['Entire_News'].str.contains('Get the latest entertainment news from India &')==True]

,Date,Time,Category,Sub-Category,Headline,Entire_News,Author,News_Link
720,2022-04-05,15:36:38,entertainment-news,bollywood-news,"Ranbir Kapoor, Alia Bhatt's Wedding Festivitie...",With reports of Alia Bhatt and Ranbir Kapoor's...,Kriti Nayyar,https://www.republicworld.com/entertainment-ne...
721,2022-04-05,15:22:35,entertainment-news,bollywood-news,Ishaan Khatter Croons 'Dhadak' In Throwback Cl...,Bollywood star Ishaan Khatter has various tale...,Aditi Rathi,https://www.republicworld.com/entertainment-ne...
722,2022-04-05,14:17:29,entertainment-news,bollywood-news,Shahid Kapoor Leaves Fans In Splits With A Hil...,Shahid Kapoor and Mira Rajput never fail to gi...,Aditi Rathi,https://www.republicworld.com/entertainment-ne...
723,2022-04-05,14:09:13,entertainment-news,bollywood-news,'Boycott snooty western awards' | Kangana Rana...,"After the Oscars, as Grammys 2022 failed to pa...",Nehal Gautam,https://www.republicworld.com/entertainment-ne...
724,2022-04-05,08:33:32,entertainment-news,bollywood-news,'He’d be nothing without her'| Babil Khan Open...,Actor Irrfan Khan's demise in April 2020 has l...,Sneha Biswas,https://www.republicworld.com/entertainment-ne...
...,...,...,...,...,...,...,...,...
955,2022-03-27,18:53:32,entertainment-news,regional-indian-cinema,'Vaashi': Tovino Thomas & Keerthy Suresh Tease...,After piquing fans' excitement with their firs...,Kriti Nayyar,https://www.republicworld.com/entertainment-ne...
956,2022-03-27,17:10:37,entertainment-news,regional-indian-cinema,Ram Charan's Wife Upasana Throws Confetti At S...,Celebrated filmmaker SS Rajamouli's critically...,Swati Singh,https://www.republicworld.com/entertainment-ne...
957,2022-03-27,16:56:03,entertainment-news,regional-indian-cinema,Chiranjeevi Posts Sweet Childhood Pic Of Ram C...,"Ram Charan, who is currently basking in the su...",Adelle Fernandes,https://www.republicworld.com/entertainment-ne...
958,2022-03-27,16:07:51,entertainment-news,regional-indian-cinema,'KGF: Chapter 2' Trailer: Here's How Fans Are ...,"After a wait of three years, fans are all set ...",Aditi Rathi,https://www.republicworld.com/entertainment-ne...


In [150]:
url = 'https://www.republicworld.com/sports-news/ipl-2022/ipl-2022-plan-is-to-not-only-hit-boundaries-and-sixes-but-choose-smart-shots-says-kl-rahul-articleshow.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
text =''

article = soup.find('h1',class_='story-title').get_text().strip()
soup.find("div",class_='width100 storytext').findAll('p')
#article = soup.find("div",class_='width100 storytext').findAll('p')


[<p>Lucknow Super Giants captain KL Rahul wants his batters to put pressure on the opposition by playing aggressive but risk-free cricket, look for the fours and sixes, but "choose smart shots".</p>,
 <p>Rahul's team's earned its second successive win in the IPL-15 here on Monday, beating Sunrisers Hyderabad by 12 runs.</p>,
 <p>Having scored 169 for seven after a disastrous start, LSG stopped SRH at 157 for nine.</p>,
 <p>Winning games likes these will give a lot of confidence to LSG, and Rahul touched upon that at the post-match presentation.</p>,
 <p>"What's pleasing is that we've found a way to stay in the game and give ourselves a good chance of winning. We did that again today," Rahul said.</p>,
 <p>While pacer Avesh Khan (4/24) shone bright with the ball, Rahul and Deepak Hooda did the bulk of scoring for LSG.</p>,
 <p>"Not ideal to lose three wickets early, but we need to give ourselves a little time since we bat deep and we have power at the death. That's something we need to 